In [14]:
import os

# Save original working directory
start_dir = os.getcwd()

# Candidate paths to look for requirements.txt
candidates = [
    "requirements.txt",
    "../requirements.txt",
    "/home/sagemaker-user/SageNotebook/requirements.txt"  # SageMaker Studio fallback
]

req_path = None
for path in candidates:
    if os.path.exists(path):
        req_path = path
        # If the file is in a parent directory, move there
        if path.startswith("../"):
            os.chdir("..")
        elif path.startswith("/home/sagemaker-user/SageNotebook"):
            os.chdir("/home/sagemaker-user/SageNotebook")
        break

if not req_path:
    raise FileNotFoundError(f"requirements.txt not found from {start_dir}")

print("Working dir:", os.getcwd())
print("Using requirements file:", req_path)

!pip install -r {req_path}


Working dir: /home/sagemaker-user/SageNotebook
Using requirements file: requirements.txt


In [ ]:
```python
       import os
       import pandas as pd
       from sklearn.model_selection import train_test_split
       from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
       import torch

       def load_data(data_dir):
           df = pd.read_csv(os.path.join(data_dir, 'banking_complaints.csv'))
           df = df.merge(pd.read_csv(os.path.join(data_dir, 'dept_product.csv')).rename(columns={'Product': 'Banking Product', 'Department': 'department'}), on='Banking Product', how='left')
           syn_map = {'Credit reporting': 'Credit Reports', 'Credit card': 'Credit Cards', 'Prepaid card': 'Credit Cards', 'Consumer Loan': 'Loans', 'Payday loan': 'Loans', 'Bank account or service': 'CASA', 'Money transfers': 'Remittance'}
           df['department'] = df['department'].fillna(df['Banking Product'].map(syn_map))
           df = df.dropna(subset=['department']).reset_index(drop=True)
           return df

       if __name__ == '__main__':
           data_dir = '/opt/ml/input/data/training'
           df = load_data(data_dir)
           tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
           labels = {dept: i for i, dept in enumerate(df['department'].unique())}
           df['label'] = df['department'].map(labels)

           train_texts, val_texts, train_labels, val_labels = train_test_split(df['Complaint Description'], df['label'], test_size=0.2, random_state=42)

           train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
           val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=128)

           class ComplaintDataset(torch.utils.data.Dataset):
               def __init__(self, encodings, labels):
                   self.encodings = encodings
                   self.labels = labels
               def __getitem__(self, idx):
                   item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
                   item['labels'] = torch.tensor(self.labels[idx])
                   return item
               def __len__(self):
                   return len(self.labels)

           train_dataset = ComplaintDataset(train_encodings, train_labels.tolist())
           val_dataset = ComplaintDataset(val_encodings, val_labels.tolist())

           model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(labels))
           training_args = TrainingArguments(
               output_dir='/opt/ml/model',
               num_train_epochs=3,
               per_device_train_batch_size=16,
               per_device_eval_batch_size=16,
               warmup_steps=500,
               weight_decay=0.01,
               logging_dir='/opt/ml/logs',
               logging_steps=10,
               evaluation_strategy='epoch',
               save_strategy='epoch',
               load_best_model_at_end=True
           )

           trainer = Trainer(
               model=model,
               args=training_args,
               train_dataset=train_dataset,
               eval_dataset=val_dataset
           )

           trainer.train()
           trainer.save_model('/opt/ml/model')
       ```